In [1]:
import pandas as pd
import sys
import os.path as op 
from numpy.linalg import norm
from gensim.models import FastText
import numpy as np


basename = "/home/adrian/PhD/Tweet-Classification-Diabetes-Distress/"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer

from preprocess import Preprocess
prep = Preprocess()

        
def preprocess_tweet(tweet):
    tweet = prep.replace_contractions(tweet)
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.remove_repeating_characters(tweet)
    tweet = prep.remove_repeating_words(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.replace_numbers(tweet)
    tweet = prep.remove_stopwords(tweet, include_personal_words=False, include_negations=False)
    tweet = [word for word in tweet if word not in ["diabetes", "diabetic"]]
    return tweet

def cosinus_similarity(a, b):
    return np.inner(a,b)/(norm(a)*norm(b))

model = FastText.load("/home/adrian/PhD/Data/FastText_embedding_20190703/ft_wordembeddings_dim300_minCount5_URL-User-toConstant_iter10_20190703")

data = pd.read_parquet("Tweets_groupLabel_20190715.parquet")
data["text_vec"] = data.text.map(lambda tweet: tweet_vectorizer(preprocess_tweet(tweet), model))
print(data.shape)
data.head()

[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


no embedding for a8 !!!!!!!!!!!!
no embedding for a8 !!!!!!!!!!!!
no embedding for a8 !!!!!!!!!!!!
(46407, 11)


,id,text,user_name,gender,typeDiabetes,geo_adm1_code,geo_name,user_location,place_full_name,label,text_vec
0,1.129140e+18,When I was initially diagnosed with #type2diab...,Lynda Jimenez,1,1,AZ,Phoenix,"Phoenix, AZ",None,4,"[0.091667205, 0.032091808, -0.081855804, -0.07..."
1,1.102559e+18,Later Twitter it's back to sleep for a nap wok...,Deanna Porter Weick,0,0,MI,Monroe,"Monroe, MI",None,11,"[0.16888438, -0.029450605, 0.0862785, -0.06317..."
2,1.039046e+18,Ugh not ready for this 3hour glucose test tomo...,Esmeralda,2,0,CA,Hemet,Hemet CA,None,22,"[0.07679383, -0.038904045, 0.13049835, -0.2382..."
3,9.581800e+17,@hotfunkytown They also lock up diabetic suppl...,Joyce Pride,0,0,MO,Maryville,"Maryville, Missouri",None,3,"[0.12150481, 0.11272085, 0.11174324, -0.108990..."
4,9.151822e+17,@I_TheeWed “Hey mom! I know you’re dreading th...,Hot Mess Momma🍁,2,0,OK,Oklahoma City,Illinois ✈️ Oklahoma,None,3,"[0.06801716, 0.14217816, 0.018431958, -0.32431..."


In [21]:
regions = list(data.groupby(by="geo_adm1_code")["id"].count().index)
topic_labels = {0:"DSMA_support_3", 1: "Diabetes_popstar_Nick_Jonas", 2:"Insulin_(access)", 3:"Negative_feelings_diabetes",
                4:"Diaversary", 5:"Misunderstandings_diabetes", 6:"Living_with_T1D", 7:"Access_insulin_insulinPrice",
                8:"Diagnosis", 9:"Inspiring_relatives_living_with_diabetes_Support", 10:"insulin pump",
                11:"Information_diabetes-related_events_of_the_day", 12:"Bloodsugar_palette_(beauty products)",
                13:"Inspiring_relatives_living_with_diabetes", 14:"Glucose_guardian", 15:"T1D",
                16:"DiabetesAwareness_Support_Donation", 17:"Glucosevariability", 18:"Diabetes-related_comorbidities",
                19:"Chronic_diseases_and_diabetes_comorbidities", 20:"DSMA_support_4", 21:"Confusion_between_T1D_and_T2D",
                22:"OGTT", 23:"DSMA_support_2", 24:"Diabetes_care", 25:"DSMA_support_1", 26:"Diabetes_and_entourage_1",
                27:"Life_with_and_without_diabetes", 28:"Diabetes_and_entourage_2", 29:"Insulin_use"}

topic_level1_labels = {
            0:"Diabetes online support", 1:"Miscallenous", 2:"Insulin", 3:"Feelings", 4:"Diabetes", 5:"Feelings", 
            6:"Life with diabetes", 7:"Insulin", 8:"Diabetes", 9:"Entourage", 10:"Insulin", 11:"Diabetes",
            12:"Miscellaneous", 13:"Entourage", 14:"Healthcare", 15:"Diabetes", 16:"Diabetes online support",
            17:"Diabetes", 18:"Diabetes-related complications", 19:"Diabetes-related complications",
            20:"Diabetes online support", 21:"Diabetes", 22:"OGTT", 23:"Diabetes online support",
            24:"Diabetes", 25:"Diabetes online support", 26:"Entourage", 27:"Life with diabetes",
            28:"Entourage", 29:"Insulin"}


tuples = list(zip([topic_labels[i] for i in range(0, 30)], [topic_level1_labels[i] for i in range(0, 30)]))
index = pd.MultiIndex.from_tuples(tuples, names=['Level1_topics', 'topics'])

print(index)
print()
print([topic_labels[i] for i in range(0,30)])
#matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=[topic_labels[i] for i in range(0,30)])
matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=index)
#matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=[str(i) for i in range(0,30)])

for j, group in data.groupby(by="label"):

    for i, v in group.groupby(by="geo_adm1_code")["id"].count().items():

#        matrix.loc[i, str(group.label.values[0])] = v / group.groupby(by="geo_adm1_code")["id"].count().sum()#roup.shape[0]
        matrix.loc[i, topic_labels[group.label.values[0]]] = v / group.groupby(by="geo_adm1_code")["id"].count().sum()#roup.shape[0]
       
matrix.head(55)

MultiIndex(levels=[['Access_insulin_insulinPrice', 'Bloodsugar_palette_(beauty products)', 'Chronic_diseases_and_diabetes_comorbidities', 'Confusion_between_T1D_and_T2D', 'DSMA_support_1', 'DSMA_support_2', 'DSMA_support_3', 'DSMA_support_4', 'Diabetes-related_comorbidities', 'DiabetesAwareness_Support_Donation', 'Diabetes_and_entourage_1', 'Diabetes_and_entourage_2', 'Diabetes_care', 'Diabetes_popstar_Nick_Jonas', 'Diagnosis', 'Diaversary', 'Glucose_guardian', 'Glucosevariability', 'Information_diabetes-related_events_of_the_day', 'Inspiring_relatives_living_with_diabetes', 'Inspiring_relatives_living_with_diabetes_Support', 'Insulin_(access)', 'Insulin_use', 'Life_with_and_without_diabetes', 'Living_with_T1D', 'Misunderstandings_diabetes', 'Negative_feelings_diabetes', 'OGTT', 'T1D', 'insulin pump'], ['Diabetes', 'Diabetes online support', 'Diabetes-related complications', 'Entourage', 'Feelings', 'Healthcare', 'Insulin', 'Life with diabetes', 'Miscallenous', 'Miscellaneous', 'OGTT']

Level1_topics,DSMA_support_3,Diabetes_popstar_Nick_Jonas,Insulin_(access),Negative_feelings_diabetes,Diaversary,Misunderstandings_diabetes,Living_with_T1D,Access_insulin_insulinPrice,Diagnosis,Inspiring_relatives_living_with_diabetes_Support,...,DSMA_support_4,Confusion_between_T1D_and_T2D,OGTT,DSMA_support_2,Diabetes_care,DSMA_support_1,Diabetes_and_entourage_1,Life_with_and_without_diabetes,Diabetes_and_entourage_2,Insulin_use
topics,Diabetes online support,Miscallenous,Insulin,Feelings,Diabetes,Feelings,Life with diabetes,Insulin,Diabetes,Entourage,...,Diabetes online support,Diabetes,OGTT,Diabetes online support,Diabetes,Diabetes online support,Entourage,Life with diabetes,Entourage,Insulin
AK,0.002656,0.004608,0.000000,0.004390,0.004212,0.002496,0.001343,0.003477,0.001978,0.001313,...,0.003961,0.003348,0.003976,0.000000,0.002234,0.001866,0.000000,0.002451,0.004350,0.002786
AL,0.004648,0.004608,0.000000,0.008780,0.004813,0.004992,0.002686,0.003477,0.005935,0.005909,...,0.005282,0.006696,0.011431,0.000000,0.005212,0.009328,0.011547,0.004902,0.008700,0.006079
AR,0.001992,0.004608,0.004049,0.006829,0.004813,0.004368,0.004701,0.005911,0.000989,0.003940,...,0.005722,0.005580,0.003976,0.000000,0.007446,0.005597,0.004619,0.002451,0.003806,0.004306
AZ,0.019920,0.018433,0.014170,0.028293,0.019856,0.018097,0.026192,0.017385,0.019782,0.020355,...,0.019366,0.021763,0.026839,0.004127,0.022338,0.024254,0.018476,0.017157,0.018488,0.022543
CA,0.130810,0.115207,0.050607,0.116585,0.113718,0.115445,0.111484,0.094576,0.118694,0.128693,...,0.112676,0.122210,0.092445,0.024759,0.114669,0.102612,0.145497,0.122549,0.132681,0.120061
CO,0.025232,0.023041,0.182186,0.016585,0.014440,0.027769,0.014775,0.035118,0.014837,0.028890,...,0.013644,0.021763,0.015905,0.011692,0.018615,0.011194,0.027714,0.019608,0.018488,0.023810
CT,0.014608,0.013825,0.002024,0.005366,0.008424,0.011856,0.007388,0.002434,0.005935,0.008536,...,0.016285,0.006696,0.002485,0.026823,0.006701,0.011194,0.013857,0.009804,0.006525,0.006586
DC,0.018592,0.004608,0.105263,0.013659,0.015644,0.023089,0.026864,0.049722,0.016815,0.022324,...,0.015845,0.019531,0.007952,0.004814,0.031273,0.016791,0.039261,0.022059,0.014682,0.027609
DE,0.003320,0.000000,0.000000,0.002927,0.000000,0.003744,0.003358,0.003477,0.004946,0.002626,...,0.003961,0.003906,0.001988,0.000000,0.004468,0.003731,0.002309,0.003268,0.002719,0.003293


In [22]:
matrix.sum()

Level1_topics                                     topics                        
DSMA_support_3                                    Diabetes online support           1.0
Diabetes_popstar_Nick_Jonas                       Miscallenous                      1.0
Insulin_(access)                                  Insulin                           1.0
Negative_feelings_diabetes                        Feelings                          1.0
Diaversary                                        Diabetes                          1.0
Misunderstandings_diabetes                        Feelings                          1.0
Living_with_T1D                                   Life with diabetes                1.0
Access_insulin_insulinPrice                       Insulin                           1.0
Diagnosis                                         Diabetes                          1.0
Inspiring_relatives_living_with_diabetes_Support  Entourage                         1.0
insulin pump                           

In [23]:
sociofactors = pd.read_parquet("socioEconom_environmental_factors_20160718.parquet")
print(sociofactors.shape)

sociofactors = sociofactors[sociofactors.LocationAbbr.isin(matrix.index)]
sociofactors.sort_values("LocationAbbr", inplace=True)
print(sociofactors.shape)
sociofactors.head()

(55, 14)
(51, 14)


,LocationAbbr,CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%,CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity,Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length,HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage,Mean income (dollars); Estimate; All households,Birth Rate,Fertility Rate,Average Age of Mother,Estimate; Gini Index,Unemployment-rate-Population-16-years-and-over__2017,Percentage_Diabetes_Diagnosed_2016
Alaska,AK,25.4,8.9,17.8,19.1,14.4,280.7,92674.0,14.12,71.51,28.42,0.4180,7.7,7.7
Alabama,AL,32.1,10.5,16.5,16.3,13.0,437.9,66337.0,12.09,62.05,27.29,0.4776,7.4,13.2
Arkansas,AR,31.9,11.5,15.5,14.4,10.7,421.4,63560.0,12.49,64.98,26.87,0.4732,6.3,12.1
Arizona,AZ,27.0,10.4,15.4,13.3,12.0,268.7,77451.0,11.67,60.84,28.21,0.4690,7.1,9.7
California,CA,19.6,9.3,16.2,16.6,13.3,284.6,101761.0,11.93,58.20,29.82,0.4889,7.7,9.7


In [4]:
sociofactors["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"] = pd.to_numeric(sociofactors["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"])
sociofactors["Mean income (dollars); Estimate; All households"] = pd.to_numeric(sociofactors["Mean income (dollars); Estimate; All households"])
sociofactors["Estimate; Gini Index"] = pd.to_numeric(sociofactors["Estimate; Gini Index"])
sociofactors["Unemployment-rate-Population-16-years-and-over__2017"] = pd.to_numeric(sociofactors["Unemployment-rate-Population-16-years-and-over__2017"])
print(sociofactors.dtypes)
del sociofactors["LocationAbbr"]
sociofactors.describe()


LocationAbbr                                                                                              object
CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%                   float64
CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%                                 float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification    float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity                         float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length     float64
HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage                                float64
Mean income (dollars); Estimate; All households                                                          float64
Birth Rate                                                                                      

,CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%,CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity,Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length,HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage,Mean income (dollars); Estimate; All households,Birth Rate,Fertility Rate,Average Age of Mother,Estimate; Gini Index,Unemployment-rate-Population-16-years-and-over__2017,Percentage_Diabetes_Diagnosed_2016
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,25.627451,10.480392,15.786275,14.139216,11.809804,326.550980,81659.156863,11.853333,61.042549,28.651961,0.464880,6.139216,9.374510
std,4.729485,1.898001,1.436805,2.297223,2.707490,53.985451,14026.409155,1.317203,5.881393,0.985483,0.020937,1.345077,1.607961
min,18.000000,6.600000,11.600000,8.200000,5.500000,228.000000,59557.000000,9.020000,49.670000,26.760000,0.418000,2.700000,6.200000
25%,22.150000,9.200000,15.200000,12.950000,10.450000,288.450000,72483.500000,11.235000,58.310000,28.005000,0.451250,5.050000,8.150000
50%,25.400000,10.500000,15.800000,14.300000,11.600000,313.900000,77369.000000,11.850000,60.190000,28.510000,0.464400,6.400000,9.500000
75%,28.700000,11.500000,16.550000,15.250000,13.450000,360.550000,91690.000000,12.440000,64.710000,29.365000,0.477850,7.200000,10.300000
max,36.800000,15.300000,19.400000,20.000000,22.600000,456.700000,121095.000000,15.660000,76.370000,30.760000,0.531400,8.800000,13.200000


In [15]:
for factor in sociofactors.columns:
    print(factor)

CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%
CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity
Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length
HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage
Mean income (dollars); Estimate; All households
Birth Rate
Fertility Rate
Average Age of Mother
Estimate; Gini Index
Unemployment-rate-Population-16-years-and-over__2017
Percentage_Diabetes_Diagnosed_2016


In [138]:
#Preprocess scale  / normalise values
from sklearn.preprocessing import MinMaxScaler

def normalise(data):
    
    min_max_scaler = MinMaxScaler()
    data["CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%"] /= 100
    data["CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length"] /= 100
    data["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"] = min_max_scaler.fit_transform(data["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"].values.reshape(-1, 1))
    data["Mean income (dollars); Estimate; All households"] = min_max_scaler.fit_transform(data["Mean income (dollars); Estimate; All households"].values.reshape(-1, 1))
    data["Birth Rate"] /= 100
    data["Fertility Rate"] /= 100
    data["Average Age of Mother"] = min_max_scaler.fit_transform(data["Average Age of Mother"].values.reshape(-1, 1))
    data["Unemployment-rate-Population-16-years-and-over__2017"] /= 100
    data["Percentage_Diabetes_Diagnosed_2016"] /= 100
    
    return data
    
sociofactors_normalised = normalise(sociofactors)
sociofactors_normalised.head()

,CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%,CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity,Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length,HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage,Mean income (dollars); Estimate; All households,Birth Rate,Fertility Rate,Average Age of Mother,Estimate; Gini Index,Unemployment-rate-Population-16-years-and-over__2017,Percentage_Diabetes_Diagnosed_2016
Alaska,0.254,0.089,0.178,0.191,0.144,0.230433,0.538155,0.1412,0.7151,0.4150,0.4180,0.077,0.077
Alabama,0.321,0.105,0.165,0.163,0.130,0.917796,0.110176,0.1209,0.6205,0.1325,0.4776,0.074,0.132
Arkansas,0.319,0.115,0.155,0.144,0.107,0.845649,0.065049,0.1249,0.6498,0.0275,0.4732,0.063,0.121
Arizona,0.270,0.104,0.154,0.133,0.120,0.177962,0.290780,0.1167,0.6084,0.3625,0.4690,0.071,0.097
California,0.196,0.093,0.162,0.166,0.133,0.247486,0.685820,0.1193,0.5820,0.7650,0.4889,0.077,0.097


In [148]:
from sklearn.preprocessing import StandardScaler

sociofactors_standardscaled = StandardScaler().fit_transform(sociofactors)
sociofactors_standardscaled

array([[-4.85706614e-02, -8.40946780e-01,  1.41547630e+00,
         2.18095831e+00,  9.66197413e-01, -8.57772218e-01,
         7.93107212e-01,  1.73794155e+00,  1.79746647e+00,
        -2.37719970e-01, -2.26144466e+00,  1.17191449e+00,
        -1.05174966e+00],
       [ 1.38217029e+00,  1.04335829e-02,  5.01687804e-01,
         9.49966820e-01,  4.43968077e-01,  2.08309822e+00,
        -1.10324886e+00,  1.81461545e-01,  1.72999077e-01,
        -1.39577592e+00,  6.13576122e-01,  9.46659566e-01,
         2.40276767e+00],
       [ 1.33946160e+00,  5.42546309e-01, -2.01226427e-01,
         1.14651168e-01, -4.13980118e-01,  1.77441907e+00,
        -1.30320257e+00,  4.88157113e-01,  6.76137498e-01,
        -1.82620379e+00,  4.01326266e-01,  1.20724859e-01,
         1.71186420e+00],
       [ 2.93098819e-01, -4.27776898e-02, -2.71517850e-01,
        -3.68952630e-01,  7.09471227e-02, -1.08226614e+00,
        -3.03002005e-01, -1.40568802e-01, -3.47816361e-02,
        -4.52933907e-01,  1.98724130e

In [7]:
from sklearn.linear_model import ElasticNetCV
from scipy import stats


def get_summary(model, X, y):

    params = np.append(model.intercept_, model.coef_)
    predictions = model.predict(X)

    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    return pd.DataFrame({"Coefficients": params, "StandardErrors":sd_b, "t_values":ts_b, "p_values": p_values})


for i in range(0,29):

    #topic = str(i)
    topic = topic_labels[i]
    print("Topic:", topic)
    y =  matrix[topic].values
    X = sociofactors.values
    #X = sociofactors_normalised.values
    #X = sociofactors_standardscaled

    print("y shape:", y.shape, "   X.shape:", X.shape)
    regr = ElasticNetCV(l1_ratio=[.0001, .001, .01, .1, .5, .7, .9, .95, .99, 1], normalize=True, 
                    cv=3, random_state=0, n_jobs=3, verbose=0, n_alphas=100, max_iter=1000)
    regr.fit(X, y)  

    print("alpha:", regr.alpha_) 
    #print(regr.coef_)
    print("intercept:", regr.intercept_) 
    #print(regr.mse_path_)
    print("l1 ratio:", regr.l1_ratio_)
    print("Score R2:", regr.score(X,y))
    #print(regr.predict([[0, 0]])) 

    new = get_summary(regr, X, y)
    print(new.head(20))
    print()
    print("=============================================")
    print()

Topic: DSMA_support_3
y shape: (51,)    X.shape: (51, 13)
alpha: 0.0009145691656889807
intercept: -0.16357643225476692
l1 ratio: 0.5
Score R2: 0.31405395411728476
    Coefficients  StandardErrors  t_values  p_values
0        -0.1636           0.404    -0.405     0.687
1        -0.0009           0.001    -0.902     0.371
2         0.0000           0.002     0.000     1.000
3         0.0000           0.005     0.000     1.000
4         0.0000           0.003     0.000     1.000
5         0.0000           0.002     0.000     1.000
6        -0.0000           0.000    -0.000     1.000
7         0.0000           0.000     0.000     1.000
8         0.0000           0.007     0.000     1.000
9        -0.0000           0.002    -0.000     1.000
10        0.0021           0.014     0.147     0.884
11        0.3124           0.274     1.139     0.260
12        0.0003           0.004     0.077     0.939
13        0.0000           0.005     0.000     1.000


Topic: Diabetes_popstar_Nick_Jonas
y sha

alpha: 0.01847469860749334
intercept: -0.13976584067909345
l1 ratio: 0.001
Score R2: 0.3116350669158525
    Coefficients  StandardErrors  t_values  p_values
0        -0.1398           0.412    -0.339     0.736
1        -0.0008           0.001    -0.775     0.442
2         0.0005           0.002     0.242     0.810
3         0.0003           0.005     0.054     0.957
4         0.0002           0.003     0.064     0.949
5         0.0001           0.002     0.058     0.954
6        -0.0000           0.000    -0.129     0.898
7         0.0000           0.000     0.180     0.858
8         0.0008           0.008     0.110     0.913
9        -0.0001           0.002    -0.063     0.950
10        0.0020           0.015     0.137     0.891
11        0.1683           0.280     0.601     0.551
12        0.0014           0.004     0.335     0.739
13        0.0013           0.005     0.278     0.782


Topic: insulin pump
y shape: (51,)    X.shape: (51, 13)
alpha: 0.020477486869017552
intercept: -0.1

alpha: 0.0005237008851500088
intercept: -0.1335173784418161
l1 ratio: 1.0
Score R2: 0.2968494304874386
    Coefficients  StandardErrors  t_values  p_values
0        -0.1335           0.419    -0.319     0.751
1        -0.0009           0.001    -0.867     0.390
2         0.0000           0.002     0.000     1.000
3         0.0000           0.005     0.000     1.000
4         0.0000           0.003     0.000     1.000
5         0.0000           0.002     0.000     1.000
6        -0.0000           0.000    -0.000     1.000
7         0.0000           0.000     0.000     1.000
8         0.0000           0.008     0.000     1.000
9        -0.0000           0.002    -0.000     1.000
10        0.0004           0.015     0.025     0.980
11        0.3579           0.285     1.257     0.215
12        0.0000           0.004     0.000     1.000
13        0.0000           0.005     0.000     1.000


Topic: Chronic_diseases_and_diabetes_comorbidities
y shape: (51,)    X.shape: (51, 13)
alpha: 0.0058

alpha: 0.016189179920200285
intercept: -0.1292807589751172
l1 ratio: 0.01
Score R2: 0.29019862997972745
    Coefficients  StandardErrors  t_values  p_values
0        -0.1293           0.403    -0.320     0.750
1        -0.0008           0.001    -0.730     0.469
2         0.0003           0.002     0.142     0.887
3         0.0000           0.005     0.000     1.000
4         0.0000           0.003     0.000     1.000
5         0.0000           0.002     0.001     0.999
6        -0.0000           0.000    -0.002     0.999
7         0.0000           0.000     0.129     0.898
8         0.0003           0.007     0.034     0.973
9        -0.0000           0.002    -0.000     1.000
10        0.0020           0.014     0.142     0.888
11        0.1787           0.274     0.652     0.517
12        0.0010           0.004     0.253     0.801
13        0.0007           0.005     0.145     0.885


Topic: Diabetes_and_entourage_2
y shape: (51,)    X.shape: (51, 13)
alpha: 0.0005704554140052168
in

In [150]:
import statsmodels.api as sm
from scipy import stats

for i in range(0,29):

    topic = str(i)
    print("Topic:", i)
    y =  matrix[topic].values
    #X = sociofactors.values
#    X = sociofactors_normalised.values
    X = sociofactors_standardscaled

    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())
    print()
    print("=================================")
    print()
    print()
    print()

Topic: 0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     2.773
Date:                Fri, 19 Jul 2019   Prob (F-statistic):            0.00749
Time:                        11:59:07   Log-Likelihood:                 136.50
No. Observations:                  51   AIC:                            -245.0
Df Residuals:                      37   BIC:                            -218.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0196      0.003      7.164

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     1.836
Date:                Fri, 19 Jul 2019   Prob (F-statistic):             0.0737
Time:                        11:59:07   Log-Likelihood:                 124.94
No. Observations:                  51   AIC:                            -221.9
Df Residuals:                      37   BIC:                            -194.8
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0196      0.003      5.711      0.0

# Further regression

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lars, LinearRegression, LassoLars, LassoLarsCV
from sklearn.metrics import r2_score, mean_squared_error

for i in range(0,29):

    topic = str(i)
    topic = topic_labels[i]
    print("Topic:", topic)
    y =  matrix[topic].values
    X = sociofactors.values
    #X = sociofactors_normalised.values
    #X = sociofactors_standardscaled

    print("y shape:", y.shape, "   X.shape:", X.shape)
    #model = LassoLars()
    #regr = GridSearchCV(model, {"alpha":[0.0, 0.01, 0.1, 1.0, 10], "normalize":[True, False], "max_iter":[300, 500, 700]},
    #                    cv=10, n_jobs=3, verbose=1)#, scoring="r2")
    
    model = LinearRegression(n_jobs=3)
    regr = model
    #regr = LassoLarsCV(verbose=1, max_iter=1000, normalize=True, cv=3, n_jobs=3)
    #regr = ElasticNetCV(l1_ratio=[.0001, .001, .01, .1, .5, .7, .9, .95, .99, 1], normalize=True, 
    #                cv=5, random_state=0, n_jobs=3, verbose=0, n_alphas=100, max_iter=1000)
    regr.fit(X, y)  

    #print(dir(regr.best_estimator_))
    #print("Best score: {} \n Best params: {}".format(regr.best_score_, regr.best_params_))
    #print("alpha:", regr.alpha_) 
    #print(regr.coef_)
    #print("intercept:", regr.intercept_) 
    #print(regr.mse_path_)
    #print("l1 ratio:", regr.l1_ratio_)
    print("Score R2:", regr.score(X,y))
    #print(regr.predict([[0, 0]])) 

    new = get_summary(regr, X, y)
#    new = get_summary(regr.best_estimator_, X, y)
    print(new.head(20))
    print()
    print("=============================================")
    print()

Topic: DSMA_support_3
y shape: (51,)    X.shape: (51, 13)
Score R2: 0.49345179759517543
    Coefficients  StandardErrors  t_values  p_values
0        -0.8094           0.347    -2.332     0.024
1        -0.0021           0.001    -2.306     0.025
2         0.0008           0.002     0.460     0.648
3         0.0042           0.004     1.061     0.294
4         0.0004           0.003     0.157     0.876
5        -0.0028           0.002    -1.510     0.137
6        -0.0001           0.000    -1.134     0.262
7        -0.0000           0.000    -1.593     0.118
8         0.0102           0.006     1.585     0.119
9        -0.0011           0.002    -0.651     0.518
10        0.0256           0.012     2.080     0.043
11        0.1310           0.236     0.556     0.581
12        0.0044           0.003     1.273     0.209
13        0.0092           0.004     2.377     0.021


Topic: Diabetes_popstar_Nick_Jonas
y shape: (51,)    X.shape: (51, 13)
Score R2: 0.500712879461189
    Coefficients

    Coefficients  StandardErrors  t_values  p_values
0        -0.1197           0.435    -0.275     0.785
1        -0.0028           0.001    -2.458     0.017
2         0.0010           0.002     0.457     0.650
3         0.0045           0.005     0.906     0.369
4        -0.0027           0.004    -0.765     0.448
5         0.0006           0.002     0.256     0.799
6        -0.0002           0.000    -1.329     0.190
7        -0.0000           0.000    -0.569     0.572
8         0.0103           0.008     1.281     0.206
9        -0.0022           0.002    -1.103     0.275
10        0.0035           0.015     0.225     0.823
11        0.1404           0.296     0.475     0.637
12        0.0004           0.004     0.101     0.920
13        0.0106           0.005     2.183     0.034


Topic: DSMA_support_2
y shape: (51,)    X.shape: (51, 13)
Score R2: 0.3345389544854862
    Coefficients  StandardErrors  t_values  p_values
0        -0.4185           0.832    -0.503     0.617
1        -

In [13]:
import statsmodels.api as sm
from scipy import stats

for i in range(0,29):
    topic = topic_labels[i]
    print("Topic:", topic)
    y =  matrix[topic].values
    X = sociofactors.values #    X = sociofactors_normalised.values #    X = sociofactors_standardscaled

    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())
    print("=================================")
    print()
    print()

Topic: DSMA_support_3
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     2.773
Date:                Mon, 22 Jul 2019   Prob (F-statistic):            0.00749
Time:                        16:39:19   Log-Likelihood:                 136.50
No. Observations:                  51   AIC:                            -245.0
Df Residuals:                      37   BIC:                            -218.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8094      0.3

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     4.471
Date:                Mon, 22 Jul 2019   Prob (F-statistic):           0.000162
Time:                        16:39:19   Log-Likelihood:                 145.90
No. Observations:                  51   AIC:                            -263.8
Df Residuals:                      37   BIC:                            -236.7
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8668      0.289     -3.003      0.0